In [1]:
import os
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig
from azureml.core.dataset import Dataset
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.conda_dependencies import CondaDependencies
import json

from dotenv import load_dotenv 
load_dotenv()

print("Azure ML SDK Version: ", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (azure-mgmt-authorization 4.0.0 (/Users/kuko/miniconda3/envs/dp/lib/python3.11/site-packages), Requirement.parse('azure-mgmt-authorization<4,>=0.40.0')).


Azure ML SDK Version:  1.49.0


### Prihlasenie do azure

In [2]:
with open('../azure/config_siemens.json', 'r') as f:
    conf = json.load(f)
    ws = Workspace.from_config(path='config_siemens.json', auth=InteractiveLoginAuthentication(tenant_id=conf['directory_id']))

### Nacitanie datasetu

In [3]:
# Dataset.get_all(workspace=ws)

In [4]:
dataset = Dataset.get_by_name(workspace=ws, name='povinec-vs')
# dataset

Request failed with: Error occurred in request., RetryError: HTTPSConnectionPool(host='westeurope.experiments.azureml.net', port=443): Max retries exceeded with url: /dataset/v1.0/subscriptions/bc82dccd-f19d-42cb-9ce3-0d5df33ef086/resourceGroups/a0047-STUFIIT-ML01/providers/Microsoft.MachineLearningServices/workspaces/a0047stufiitml01/datasets/query/name=povinec-vs?includeLatestDefinition=true (Caused by ResponseError('too many 503 error responses'))
2024-01-21 02:32:17.907250 | ActivityCompleted: Activity=get_by_name, HowEnded=Failure, Duration=72091.52 [ms], Info = {'activity_id': '341fb10f-b959-40b7-9b68-7afe77faa446', 'activity_name': 'get_by_name', 'activity_type': 'PublicApi', 'app_name': 'dataset', 'source': 'azureml.dataset', 'version': '1.49.0', 'dataprepVersion': '', 'sparkVersion': '', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': ''}, Exception=ClientRequestError; Error occurred in request., RetryError: HTTPSC

ClientRequestError: Error occurred in request., RetryError: HTTPSConnectionPool(host='westeurope.experiments.azureml.net', port=443): Max retries exceeded with url: /dataset/v1.0/subscriptions/bc82dccd-f19d-42cb-9ce3-0d5df33ef086/resourceGroups/a0047-STUFIIT-ML01/providers/Microsoft.MachineLearningServices/workspaces/a0047stufiitml01/datasets/query/name=povinec-vs?includeLatestDefinition=true (Caused by ResponseError('too many 503 error responses'))

## Experimenty

In [ ]:
config = ScriptRunConfig(
    source_directory='../src/',
    script='main.py',
    arguments=[
        '--data-path', dataset.as_mount(), 
        '--wandb', os.getenv('WANDB'),
        '--model-path', './models/pretrained.pt'
    ], 
    compute_target='V100x1-xpovinec'
)

Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'


### Vytvorenie experimentu

In [ ]:
experiment = Experiment(workspace=ws, name='xpovinec-VS')

### Nastavenie env

In [ ]:
env = Environment.get(workspace=ws, name='xpovinec-env')
config.run_config.environment = env

### Spustenie experimentu

In [ ]:
run = experiment.submit(config)

In [ ]:
# toto zisti, kde bezi a vrati to stranku
aml_url = run.get_portal_url()
print(aml_url)

https://ml.azure.com/runs/xpovinec-VS_1705573008_208f1032?wsid=/subscriptions/bc82dccd-f19d-42cb-9ce3-0d5df33ef086/resourcegroups/a0047-STUFIIT-ML01/workspaces/a0047stufiitml01&tid=5dbf1add-202a-4b8d-815b-bf0fb024e033
